**Import needed packages/modules**

In [ ]:
# Cell 1
import matplotlib.pyplot as plt
import numpy as np
from numba import int64, float64, vectorize

**Declare a numba accelerated function that computes the Halton QRNG**
1. The parameter $n$ is an integer of any size
2. The parameter $p$ is a prime number

In [ ]:
# Cell 2
@vectorize([float64(int64, int64)], nopython=True)
def halton(n, p):
    h, f = 0, 1
    while n > 0:
        f = f / p
        h += (n % p) * f
        n = int(n / p)
    return h

**Set the total number of samples to take**

In [ ]:
# Cell 3
total_dots = 6_250_000

**Create `total_dots` samples of 4D Cartesian points $(x,y,z,w)$ using the Halton sequence**
1. The Halton QRNG returns a random float [0,1)
2. Subtract that float from 1, so the interval flips to (0,1] ensuring any points on the perimeter will now contribute to the "content"
3. Scale the result so it now falls in the interval [-1, 1]

In [ ]:
# Cell 4
x = (1 - halton(np.arange(total_dots), 2)) * 2 - 1
y = (1 - halton(np.arange(total_dots), 3)) * 2 - 1
z = (1 - halton(np.arange(total_dots), 5)) * 2 - 1
w = (1 - halton(np.arange(total_dots), 7)) * 2 - 1

**Create an array $d$ that contains the distance from the origin $(0,0,0,0)$ for every point $(x,y,z,w)$**\
Leverage the fact the exponentiation and addition operators are "vector aware"

In [ ]:
# Cell 5
d = x**2 + y**2 + z**2 + w**2

**Calculate the absolute percent error in the content estimation**
1. The actual/expected content of a unit hypersphere is exactly $\large\frac{\pi^2}{2}$
2. The observed/estimated content using the Monte Carlo formulation $\large=16\times\frac{dots_{\ inside}}{dots_{\ total}}$


In [ ]:
# Cell 7
act = np.pi**2 / 2
est = 16 * np.count_nonzero(d <= 1.0) / total_dots
err = np.abs((est - act) / act)

print(f"dots = {total_dots:,}")
print(f"act = {act:.6f}")
print(f"est = {est:.6f}")
print(f"err = {err:.5%}")